In [1]:
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages")

from datetime import datetime
import pandas as pd
import numpy as np

In [13]:
def mcvalue(m, mbin):
    density = np.histogram(m, range = (min(m) - 1, max(m) + 1), density=True)
    return density[1][np.argmax(density[0])]

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    """
    To calculate the L2 distance on the sphere
    """
    rad=np.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(np.sin(rad*dlat/2))**2 + np.cos(rad*lat1)*np.cos(rad*lat2)*(np.sin(rad*dlon/2))**2
    distancia=2*R*np.arcsin(np.sqrt(a))
    return distancia

In [4]:
def dist_quakes(catalogo, B, c, d):
    """
    Returns matrix with the distances between earthquakes. If the value is negative
    then earthquake i happened after earthquake j.
    The geographical distance used is the product metric (haversine^2 + L2^2)^1/2 (which
    won't be truthful for long distances)
    """
    distancias = np.zeros(shape=(catalogo.shape[1], catalogo.shape[1]))
    for i in range(catalogo.shape[1]):
        for j in range(i+1, catalogo.shape[1]):
            sismo_i = catalogo.iloc[i,:]
            sismo_j = catalogo.iloc[j,:]
            
            tau = sismo_j["timestamp"] - sismo_i["timestamp"]
            tau = tau.total_seconds() # The time scale will be in seconds
            
            r = np.sqrt(haversine(sismo_i["Longitud"], sismo_j['Longitud'],
                        sismo_i['Latitud'], sismo_j['Latitud'])**2 + 
                        (sismo_j['Profundidad']-sismo_i['Profundidad'])**2 )
            delta_m = sismo_j["Magnitud"] - sismo_i["Magnitud"]
            
            distancias[i,j] = c*tau* (r**d) *(10**(-B * delta_m))
             
    return distancias           

In [6]:
sismos = pd.read_csv("SSNMX_cat.csv")
sismos

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,2017-09-01,01:26:16,3.4,15.9832,-96.9505,25.8,"18 km al NORESTE de PUERTO ESCONDIDO, OAX",2017-09-01,06:26:16,revisado
1,2017-09-01,01:29:47,3.5,16.6938,-95.1257,94.8,"15 km al NORTE de CD IXTEPEC, OAX",2017-09-01,06:29:47,revisado
2,2017-09-01,02:10:55,3.8,15.5425,-92.4787,187.6,"32 km al NOROESTE de MOTOZINTLA, CHIS",2017-09-01,07:10:55,revisado
3,2017-09-01,02:15:28,3.5,16.0508,-97.1588,26.3,"23 km al NOROESTE de PUERTO ESCONDIDO, OAX",2017-09-01,07:15:28,revisado
4,2017-09-01,02:24:18,3.6,17.1730,-94.9790,121.0,"34 km al NORTE de MATIAS ROMERO, OAX",2017-09-01,07:24:18,revisado
...,...,...,...,...,...,...,...,...,...,...
54557,2021-05-10,02:10:19,3.8,15.9500,-95.6600,54.0,"54 km al NORESTE de CRUCECITA, OAX",2021-05-10,07:10:19,verificado
54558,2021-05-10,02:15:51,3.9,17.4100,-94.8400,144.0,"53 km al SUR de SAYULA DE ALEMAN, VER",2021-05-10,07:15:51,verificado
54559,2021-05-10,03:35:19,3.9,14.9200,-94.0200,14.0,"121 km al SUROESTE de PIJIJIAPAN, CHIS",2021-05-10,08:35:19,verificado
54560,2021-05-10,05:13:42,4.5,14.0000,-92.1500,43.0,"75 km al SUR de CD HIDALGO, CHIS",2021-05-10,10:13:42,verificado


In [7]:
sismos["timestamp"] = sismos[["Fecha UTC", "Hora UTC"]].agg(' '.join, axis=1)
sismos["timestamp"] = sismos["timestamp"].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
sismos.drop(['Fecha UTC','Hora UTC', 'Estatus'], 1)

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,timestamp
0,2017-09-01,01:26:16,3.4,15.9832,-96.9505,25.8,"18 km al NORESTE de PUERTO ESCONDIDO, OAX",2017-09-01 06:26:16
1,2017-09-01,01:29:47,3.5,16.6938,-95.1257,94.8,"15 km al NORTE de CD IXTEPEC, OAX",2017-09-01 06:29:47
2,2017-09-01,02:10:55,3.8,15.5425,-92.4787,187.6,"32 km al NOROESTE de MOTOZINTLA, CHIS",2017-09-01 07:10:55
3,2017-09-01,02:15:28,3.5,16.0508,-97.1588,26.3,"23 km al NOROESTE de PUERTO ESCONDIDO, OAX",2017-09-01 07:15:28
4,2017-09-01,02:24:18,3.6,17.1730,-94.9790,121.0,"34 km al NORTE de MATIAS ROMERO, OAX",2017-09-01 07:24:18
...,...,...,...,...,...,...,...,...
54557,2021-05-10,02:10:19,3.8,15.9500,-95.6600,54.0,"54 km al NORESTE de CRUCECITA, OAX",2021-05-10 07:10:19
54558,2021-05-10,02:15:51,3.9,17.4100,-94.8400,144.0,"53 km al SUR de SAYULA DE ALEMAN, VER",2021-05-10 07:15:51
54559,2021-05-10,03:35:19,3.9,14.9200,-94.0200,14.0,"121 km al SUROESTE de PIJIJIAPAN, CHIS",2021-05-10 08:35:19
54560,2021-05-10,05:13:42,4.5,14.0000,-92.1500,43.0,"75 km al SUR de CD HIDALGO, CHIS",2021-05-10 10:13:42


In [14]:
m = sismos['Magnitud'].values
m = m[~np.isnan(m)]
mbin = 0.1 
Mc = mcvalue(m, mbin)
# Aki ML estimates
B = np.log10(np.e)/(m[m>Mc-mbin/2].mean() - (Mc-mbin/2))
#d = len(m[m>0])
d = 2*B

#Arbitrary c. (We don't know what it is)
#We're also not entirely sure about the d. We need to find more sources.
dist_quakes(sismos, B, 1, d)

array([[0.00000000e+00, 1.84415383e+07, 1.53783276e+08, 2.57528829e+08,
        2.63968043e+08, 2.00889471e+08, 6.40764749e+08, 5.69130096e+08,
        7.10318855e+08, 4.11219240e+08, 8.67175075e+08],
       [0.00000000e+00, 0.00000000e+00, 1.65811835e+08, 2.80419195e+08,
        2.90671715e+08, 2.28420935e+08, 7.32261263e+08, 6.50353325e+08,
        8.13865207e+08, 4.71390878e+08, 9.97462431e+08],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.27168851e+07,
        1.09072398e+08, 2.24649112e+08, 7.83844748e+08, 7.01912552e+08,
        9.08712761e+08, 5.30420351e+08, 1.18870064e+09],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.62934548e+07, 1.35772346e+08, 4.80335854e+08, 4.31420345e+08,
        5.60525134e+08, 3.27575921e+08, 7.41032624e+08],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.39842833e+08, 5.12004239e+08, 4.61082728e+08,
        6.06687808e+08, 3.55508470e+08, 8.19356455e+